In [1]:
!pip install transformers
!pip install wandb -qU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
     |████████████████████████████████| 163 kB 62.8 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 162 kB 41.6 MB/s 
     |████████████████████████████████| 181 kB 56.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 162 kB 47.1 MB/s 
     |████████████████████████████████| 158 kB 47.6 MB/s 
     |████████████████████████████████| 157 kB 45.2 MB/s 
     |████████████████████████████████| 157 kB 42.8 MB/s 
     |████████████████████████████████| 157 kB 47.8 MB/s 
     |████████████████████████████████| 157 kB 55.0 MB/s 
     |████████████████████████████████| 157 kB 57.0 MB/s 
     |████████████████████████████████| 157 kB 50.9 MB/s 
     |█████████████████████████████

In [2]:
import os
import random

import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline
import wandb
wandb.login()
#5f453a4860d4bd53ede533c4c7e613f5513ba188

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained('youscan/ukr-roberta-base')
    model = AutoModel.from_pretrained('youscan/ukr-roberta-base')
    return tokenizer, model

tokenizer, model = load_model()

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at youscan/ukr-roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def model_embedding(text, tokenizer, model):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    outputs = model(input_ids, output_hidden_states=True)
    emb = outputs[2]
    emb = np.array([i[0].detach().numpy() for i in emb])[:]
    emb = emb.mean(axis=(0, 1))
    return emb

In [6]:
def load_data():
    data = pd.read_csv('train_small2.csv', index_col=0)
    data['text'] = data['text'].str.lower()
    return data

def train_test_split(data):
    train_data = data[data['target_numerical'] != 2].copy()
    test_data = data[data['target_numerical'] == 2].copy()
    return train_data, test_data

data = load_data()
train_data, test_data = train_test_split(data)

In [7]:
labels_list = train_data['target'].values
descriptions_list = train_data['target'].values
embeddings_list = {}

In [8]:
for label, descr in zip(labels_list, descriptions_list):
    tmp = embeddings_list.get(label, [])
    tmp.append(model_embedding(descr, tokenizer, model))
    embeddings_list[label] = tmp

In [9]:
for label, embeddings in embeddings_list.items():
    embeddings_list[label] = np.mean(embeddings, axis=0)

In [10]:
test_labels_list = test_data['target'].values
test_descriptions_list = test_data['text'].values

In [14]:
train_labels_list = train_data['target'].values
train_descriptions_list = train_data['text'].values

In [11]:
def distance(a, b):
    return sum([(i - j) ** 2 for i, j in zip(a, b)]) ** .5

In [20]:
t = 0
pred_labels = []
correctly_predicted = 0
amount_of_prediction = 0
for ind in range(len(train_descriptions_list)):

    wandb.init(
        # Set the project where this run will be logged
        project="project-test", 
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=f"experiment_{ind}", 
        # Track hyperparameters and run metadata
        config={
        "learning_rate": 0.02,
        "architecture": "roberta",
        "dataset": "ukrainian news classification",
    })
    label = train_labels_list[ind]
    descr = train_descriptions_list[ind]
    print(f'LABEL {label}\tDESCR: {descr}')

    train_emb = model_embedding(descr, tokenizer, model)

    scores = list((distance(el, train_emb), k) for k, el in embeddings_list.items())
    if scores:
        sorted_scores = sorted(scores, key=lambda x: x[0])
        best_preds = sorted_scores[0][1]
        amount_of_prediction += 1
        if label == best_preds:
            correctly_predicted += 1
        accuracy = correctly_predicted / amount_of_prediction
        #print(label, best_preds)
        wandb.log({"acc": accuracy})
        wandb.log({"actual/predicted": (label, best_preds)})
        #wandb.log({"acc": label, "loss": loss})
        pred_labels.append(best_preds)
        if label == best_preds:
            t += 1

print(t / len(test_descriptions_list))
# Mark the run as finished
wandb.finish()

acc,▁
acc,0.0


LABEL новини	DESCR: про це на своїй сторінці у facebook написав президент компанії петро чернишов. він зазначив, що сьогодні вранці з обшуками до головного офісу прийшло близько трьох десятків податкових поліцейських. вони мали постанову шевченківського районного суду києва щодо дозволу провести обшук офісу. "що ж вони шукають в такому великому складі? три угоди, укладені дуже давно ще минулим керівництвом компанії, які вже припинили дію, і кілька десятків додаткових документів до них", - наголосив чернишов. він нагадав, що "київстар" вже майже два роки судиться з дфс. остання вважає, що компанія недоплатила до бюджету майже півтора мільярда гривень. "ми вважаємо, що цю справу вигадали, адже компанія завжди справно і в повному обсязі перераховує всі податки і збори до бюджету країни. за 20 років роботи "київстар" сплатив до скарбниці 52 млрд грн різних податків і зборів", - зауважив чернишов. при цьому від подякував службовцям податкової за те, що прийшли на обшук "без зброї та масок",

acc,▁
acc,0.0


LABEL новини	DESCR: про це повідомляється в доповіді некомерційної організації oxfam, діяльність якої присвячена боротьбі з бідністю, передає еспресо.tv. такою ж сумою, як і найбідніша половина людства, володіють загалом 62 найбагатших людей у світу. їх загальний статок виріс за той же період на $1,76 трлн. а ще в 2010 році сумою, порівняно з тією, якою володіла бідніша половина людства, володіли 388 найбагатших людей. також в oxfarm зазначають, що сукупні статки 1% найбільш забезпечених людей світу вперше можуть бути більшими, ніж сукупний статок 99% інших жителів планети до 2016 року. в даний час 1% найбагатших людей належить 48% світового добробуту, а в наступному році даний показник може вперше перевищити 50%, попереджають експерти. як зазначили в oxfam, зростаюча нерівність стримує боротьбу з глобальною бідністю, причому відбувається це в період, коли більше 1 млрд людей як і раніше живуть на менш ніж $1,25 на день. причинами зростаючої різниці в достатку між бідними і багатими ор

acc,▁
acc,0.0


...

acc,▁
acc,0.5


LABEL новини	DESCR: творці стрічки "поводир" дають змогу людям із вадами зору подивитися картину нарівні з іншими глядачами, повідомляє "радіо свобода", інформує еспресо.tv. за словами режисера стрічки олеся саніна, це поширений у світі спосіб давати людям із вадами зору додаткову інформацію про сцени на екрані. фактично, це ще одна додаткова аудіодоріжка, коментар, який начитує диктор для незрячих людей. обладнання вдалося закупити на кошти одного з найкрупніших українських банків. після демонстрації "поводиря" його залишать у кінотеатрах і використовуватимуть для інших фільмів. "це обладнання в шести найбільших містах україни буде стаціонарним. і ми довго працювали, у львові нам допомогли наші незрячі експерти – це оксана потимко, вона декілька анімаційних фільмів перед цим робила. і це стало окремою частиною нашого фільму", – говорить режисер. до зйомок "поводиря"долучилися також незрячі люди, що зіграли у стрічці сліпих бандуристів, яких переслідує радянська влада. світова прем’єра

acc,▁
acc,0.4


LABEL новини	DESCR: про це на своїй сторінці у facebook повідомили організатори заходу folk ukraine. зазначається, що всього її прикрасять 800 тематичних іграшок і понад 4 кілометри різнокольорових гірлянд. "саме лускунчик втілює перемогу добра над злом і є символом нового року та різдва в європі. ми ж поєднаємо його історію з українськими традиціями", - розповіли організатори. також folk ukraine оприлюднив програму заходів: 14 грудня — відкриття різдвяного ярмарку 19 грудня — урочисте засвічення ялинки 25 грудня — католицьке різдво 31 грудня — зустріч нового року 7 січня — святкова хода звіздарів та різдвяний концерт а в інші дні нас чекатимуть святкові концерти на великій сцені. організатори обіцяють занурити відвідувачів у чарівний світ пригод лускунчика та його королівств: королівство подарунків — ярмарок сувенірів та найкращих подарунків до свята. королівство смаку — лише найсмачніші страви та напої. королівство розваг — казкове місто для дітей. нескінченні веселощі, атракціони, а

acc,▁
acc,0.33333
